In [1]:
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import coo_array
from pathlib import Path
import networkx as nx
import pandas as pd
import numpy as np
import itertools

# 2. Custom Method

This method adapts the pagerank technique outlined in [Zhang *et al.,* 2022](https://arxiv.org/abs/2104.02764). In that work, the random walk uses relative edge weights to assign probabilities of traversing edges from a given node. This means edges are weighted *locally* - around a given node - but not *globally*. 

In this method, I use the weighted pagerank method from Zhang *et al.,*, but I adjust the gamma parameter - the probability of randomly teleporting from any node - to be *node specific* and based on the total edge suspicion of a given node.

Mainly: 

Let $M$ be a transition matrix with: 
$$m_{ij} = \begin{cases}
    \theta w_{ji}/s_j^{out} + (1-\theta)a_{ji}/d_j^{out} && \texttt{if}\; d_j^{out}\neq 0\\
    0 && \texttt{if}\; d_j^{out}=0
\end{cases}$$
Where: 
- $\theta$ is a tunable parameter that represents how important edge weights should be in the pagerank alg
- $w_{ji}$ is the weight of an edge from node $j \rightarrow i$
- $s_j^{out} = \sum_{v \in V|j \rightarrow v}w_{jv}$ is the "strength" of outgoing edges from node $j$
- $a_{ji}$ = $1 \:\texttt{if}\: j \rightarrow i \:\texttt{else}\: 0$
- $d_j^{out} = \sum_{v \in V|j \rightarrow v}a_{jv}$
- $\beta_i$ is the node importance score
  


Then the pagerank can be calculated with power iterations on
$$P=\boldsymbol{\gamma} MP + (1-\boldsymbol{\gamma})\boldsymbol{\beta}/||\boldsymbol{\beta}||_1$$

Where:
- $1-\boldsymbol{\gamma}$ is a vector of hyperparameters representing the probability of randomly teleporting from a given node
- $\boldsymbol{\beta}/||\boldsymbol{\beta}||_1$ is the vectorized version of $\beta_{i}/\sum_{i\in V}\beta_i$

---
*note the other formulation in the paper results in a dense transition matrix that is nxn.... too big*

*note that for $m_{ij}$, $i$ is the target and $j$ is the source*

In [2]:
THETA = 1
REVERSE = True

DATAPATH = Path('../data/processed/pagerank')
OUTPATH = DATAPATH.parent / 'nodes_custom_rev.parquet'

n_df = pd.read_parquet(DATAPATH / 'input_node_scores.parquet')
e_df = pd.read_parquet(DATAPATH / 'input_edge_scores.parquet')
e_df = e_df[['cust_id_sender', 'cust_id_receiver', 'score']].copy()

In [3]:
#reversing edges
if REVERSE:
    e2_df = pd.DataFrame()
    e2_df['cust_id_receiver'] = e_df['cust_id_sender'].copy()
    e2_df['cust_id_sender'] = e_df['cust_id_receiver'].copy()
    e2_df['score'] = e_df['score']

    e_df = pd.concat([e_df, e2_df])
    e_df.sample(3)

## Constructing $\boldsymbol{\beta}/||\boldsymbol{\beta}||_1$

In [4]:
# Get all nodes in the graph
node_list = []
node_list.extend(e_df.cust_id_sender.tolist() + e_df.cust_id_receiver.tolist() + n_df.cust_id.tolist())
node_list = list(set(node_list)) #hack to remove duplicate"

In [5]:
# Construct B
b_df = pd.DataFrame(data={'cust_id':node_list})
b_df = b_df.merge(n_df, on='cust_id', how='left')
b_df = b_df.rename(columns={'score':'b_i'})
b_df = b_df.fillna(0)

one_norm = abs(b_df['b_i']).sum()

b_df['b_i'] = b_df['b_i'] / one_norm

b_df.sort_values('b_i', ascending=False).head(3)

,cust_id,b_i
99473,CUST76986222,0.000672
118407,CUST60968343,0.000662
21260,CUST73079564,0.000624


## Node Encoding
This section encodes the cust_ids ordinally.

In [6]:
le = LabelEncoder()
le.fit(node_list)

e_df['cust_id_sender'] = le.transform(e_df['cust_id_sender'])
e_df['cust_id_receiver'] = le.transform(e_df['cust_id_receiver'])

b_df['cust_id'] = le.transform(b_df['cust_id'])

node_enc = le.transform(node_list)

e_df.sample(3)

,cust_id_sender,cust_id_receiver,score
146211,229329,188039,0.0
127279,10679,249564,0.0
214667,153618,110197,0.0


## Constructing $M$
*note that for $m_{ij}$, $i$ is the target node and $j$ is the source node*

In [7]:
e_df['score'] += 0.01 #No edges can have 0 weight, so we add a relatively small value to all edges

#Calculate w_ji
w_df = e_df.groupby(['cust_id_sender', 'cust_id_receiver'], as_index=False)['score'].sum()
w_df = w_df.rename(columns={'score':'w_ji'})
e_df = e_df.merge(w_df, on=['cust_id_sender', 'cust_id_receiver'], how='left')

#Calculate a_ji
a_df = e_df.groupby(['cust_id_sender', 'cust_id_receiver'], as_index=False)['score'].count()
# a_df = e_df.groupby 
a_df = a_df.rename(columns={'score':'a_ji'})
e_df = e_df.merge(a_df, on=['cust_id_sender', 'cust_id_receiver'], how='left')

#Calculate s_j
s_df = e_df.groupby(['cust_id_sender'], as_index=False)['score'].sum()
s_df = s_df.rename(columns={'score':'s_j'})
e_df = e_df.merge(s_df, on='cust_id_sender', how='left')

#Calculate d_j
d_df = e_df.groupby(['cust_id_sender'], as_index=False)['score'].count()
d_df = d_df.rename(columns={'score':'d_j'})
e_df = e_df.merge(d_df, on='cust_id_sender', how='left')

#Remove duplicate edges... taking max is just a hack
e_df = e_df.groupby(['cust_id_sender', 'cust_id_receiver'], as_index=False).max()

e_df.sample(10)

,cust_id_sender,cust_id_receiver,score,w_ji,a_ji,s_j,d_j
92823,22538,3700,0.010000,0.010000,1,0.195000,7
885602,224135,46552,0.010000,0.010000,1,0.400000,15
908509,230753,64343,0.343333,0.343333,1,0.686667,2
681854,168951,219601,0.010000,0.010000,1,0.305000,18
333676,82209,133365,0.010000,0.010000,1,0.020000,2
781247,193874,189126,0.010000,0.010000,1,0.165000,4
809426,201875,12177,0.010000,0.010000,1,0.050000,5
602934,149025,122156,0.010000,0.010000,1,0.508333,5
1040335,269048,53554,0.135000,0.135000,1,0.185000,6
462708,114004,292736,0.010000,0.010000,1,0.195000,7


In [8]:
def calc_m(r): 
    m = THETA*r.w_ji/r.s_j + (1-THETA)*r.a_ji/r.d_j
    return m

#calculate m_ij for sender j, receiver i
e_df['m'] = e_df.apply(lambda r: calc_m(r), axis=1)

In [9]:
#Verify matrix is column stochastic... m should all be 1.0
e_df.groupby('cust_id_sender').sum().sort_values('m', ascending=True).head(3)

,cust_id_receiver,score,w_ji,a_ji,s_j,d_j,m
cust_id_sender,,,,,,,
91885,354058,0.155,0.155,3,0.465,9,1.0
288107,237774,0.155,0.155,3,0.465,9,1.0
207348,249041,0.155,0.155,3,0.465,9,1.0


## Calculate $\boldsymbol{\gamma}$

In [10]:
# Calculate gammas
gamma_df = e_df.groupby(['cust_id_sender'], as_index=False)['s_j'].max() #trick, all s_j are the same
gamma_df['gamma'] = np.exp(gamma_df['s_j'])/(0.8+np.exp(gamma_df['s_j']))
gamma_df = gamma_df.rename(columns={'cust_id_sender': 'cust_id'})

# Create gamma array
gamma_df = gamma_df.merge(b_df, on='cust_id', how='right') #trick to get all node ids
gamma_df = gamma_df.fillna(np.exp(0)/(0.8+np.exp(0)))
gamma_df = gamma_df.sort_values(['cust_id'])
gamma = gamma_df['gamma'].values
gamma

array([0.57031674, 0.59101032, 0.56048818, ..., 0.56048818, 0.56048818,
       0.56048818])

## Pagerank

In [11]:
def custom_pagerank(M, B, P, gamma, tol=1e-10, maxit=1000):
    """Computes the node and edge weighted pagerank using a custom algorithm
    
    This function takes in a transition matrix, node weights, and an initial pagerank vector
    and computes the weighted pagerank, stopping when the difference between iterations is < 
    tolerence or when the maximum # iterations is reached.
    
    Args:
        M: The transition matrix according to Zhang et al. should be a scipy sparse coordinate matrix (NxN)
        B: A numpy array of relative node weights (Nx1)
        P: A numpy array vector of initial pageranks, normally just uniform (Nx1) 
        gamma: An array of probabilities of randomly teleporting from a given node (Nx1)
        tol: iteration tolerance - when the maximum RELATIVE change in a node score is below this value, iterations stop
        maxit: maximum iterations
    
    Returns
        P: The final pagerank vector, min-max scaled to be in [0-1]
    
    """
    
    itcount = 0
    max_diff = 1000 #placeholder large value
    diff_list = []
    while (itcount <= maxit) and (max_diff >= tol):
           
        #Pagerank iter
        P_int = gamma*M.dot(P) + (1-gamma)*B
                
        #Adding leaked pagerank back
        #need to do this since we don't preprocess to remove dead ends.
        leak = np.sum(P_int)
        P_int = P_int + (1-leak)/len(P)
        
        max_diff = (np.absolute(P-P_int)/P).max()
        diff_list.append(max_diff)
        itcount += 1
        P = P_int
        
    
    print(f'Final max (relative) error: {max_diff:.3}')
    print(f'Final iterations: {itcount}')
    # Scaling
    P = (P - P.min())/(P.max() - P.min())
    
    return P, diff_list

In [12]:
#Construct necessary matrices
i = e_df['cust_id_receiver'].values
j = e_df['cust_id_sender'].values
m = e_df['m'].values

N = b_df.shape[0]
B = b_df.sort_values('cust_id')['b_i'].values
M = coo_array((m,(i,j)), shape=(N,N))
P = np.full(N, 1/N)

#Run Pagerank
P, dl = custom_pagerank(M, B, P, gamma)

Final max (relative) error: 0.000145
Final iterations: 1001


## Exporting for Webapp
The pagerank data needs to be converted back into KYC-esque data, with customer names, countries, etc. but this needs to include *external customers*

In [13]:
def clean_pagerank_results(pagerank_df):
    """Takes in a [id, score] dataframe and turns it back into KYC-style data"""
    
    datapath = Path('../data/processed/')
    
    kyc_df = pd.read_parquet(datapath / 'kyc.parquet')
    wdf = pd.read_parquet(datapath / 'wire.parquet')
    edf = pd.read_parquet(datapath / 'emt.parquet')
    
    cleaned = pagerank_df.copy()
    
    # Join with kyc data and add country column
    kyc_df['country'] = 'CA'
    cleaned = pd.merge(cleaned, kyc_df, how='left', on='cust_id')

    # Get additional countries from wiretransfer data
    s1 = wdf[['cust_id_sender', 'country_sender']].copy().rename(columns={'cust_id_sender':'cust_id', 'country_sender':'country'})
    s2 = wdf[['cust_id_receiver', 'country_receiver']].copy().rename(columns={'cust_id_receiver':'cust_id', 'country_receiver':'country'})
    countries = pd.concat([s1,s2])
    countries = countries.drop_duplicates()

    cleaned = pd.merge(cleaned, countries, how='left', on='cust_id')
    cleaned['country_x'] = cleaned['country_x'].combine_first(cleaned['country_y'])
    cleaned['country'] = cleaned['country_x']
    cleaned = cleaned.drop(columns=['country_x', 'country_y'])

    return cleaned

In [14]:
P_df = pd.DataFrame(data=P, columns=['score'])
P_df['cust_id'] = P_df.index
P_df['cust_id'] = le.inverse_transform(P_df['cust_id']) #transform back to actual IDS
P_df = P_df[['cust_id','score']]
P_df = clean_pagerank_results(P_df)
P_df.to_parquet(OUTPATH)

## Vis
jw = 'CUST33059790'
mw = 'CUST48666641'

In [15]:
e_df.groupby('cust_id_sender').max().sort_values('s_j')

,cust_id_receiver,score,w_ji,a_ji,s_j,d_j,m
cust_id_sender,,,,,,,
206366,41336,0.01,0.01,1,0.010000,1,1.000000
212750,68340,0.01,0.01,1,0.010000,1,1.000000
113133,126094,0.01,0.01,1,0.010000,1,1.000000
212751,180347,0.01,0.01,1,0.010000,1,1.000000
48481,110358,0.01,0.01,1,0.010000,1,1.000000
...,...,...,...,...,...,...,...
39809,262166,10.26,10.52,2,12.390000,14,0.849072
138529,298388,10.26,10.78,3,13.200000,20,0.816667
50119,291918,10.26,21.04,4,25.761667,22,0.816717


In [16]:
kdf = pd.read_parquet('../data/processed/kyc.parquet')
idx = le.inverse_transform([138529])[0]
jwid = le.transform(['CUST33059790'])[0]

kdf[kdf.cust_id==idx]

,name,gender,occupation,age,tenure,cust_id,occ_wealth,occ_animal,occ_int,label,named_trafficker
109445,ÉLISABETH LACROIX,female,Antique Dealer,43.0,2.0,CUST73707963,1,0,1,1,0.0


In [17]:
edf = e_df.copy()
edf[(edf.cust_id_sender==138529)]

,cust_id_sender,cust_id_receiver,score,w_ji,a_ji,s_j,d_j,m
561086,138529,5549,0.135,0.135,1,13.2,20,0.010227
561087,138529,10837,0.135,0.135,1,13.2,20,0.010227
561088,138529,42601,0.135,0.135,1,13.2,20,0.010227
561089,138529,56020,0.135,0.135,1,13.2,20,0.010227
561090,138529,56074,0.135,0.135,1,13.2,20,0.010227
561091,138529,69919,0.260,0.260,1,13.2,20,0.019697
561092,138529,77469,10.260,10.780,3,13.2,20,0.816667
561093,138529,141747,0.135,0.135,1,13.2,20,0.010227
561094,138529,151801,0.135,0.135,1,13.2,20,0.010227
561095,138529,160036,0.135,0.135,1,13.2,20,0.010227


In [18]:
edf = e_df.copy()
edf[(edf.cust_id_sender==50119)]

,cust_id_sender,cust_id_receiver,score,w_ji,a_ji,s_j,d_j,m
204700,50119,17394,0.135000,0.135000,1,25.761667,22,0.005240
204701,50119,24018,0.135000,0.135000,1,25.761667,22,0.005240
204702,50119,27946,0.135000,0.135000,1,25.761667,22,0.005240
204703,50119,55177,0.260000,0.260000,1,25.761667,22,0.010093
204704,50119,71739,0.260000,0.260000,1,25.761667,22,0.010093
204705,50119,83876,0.385000,0.385000,1,25.761667,22,0.014945
204706,50119,131091,0.260000,0.260000,1,25.761667,22,0.010093
204707,50119,149249,0.260000,0.260000,1,25.761667,22,0.010093
204708,50119,164781,10.260000,21.040000,4,25.761667,22,0.816717
204709,50119,193943,0.385000,0.385000,1,25.761667,22,0.014945


In [19]:
kdf[kdf.cust_id==le.inverse_transform([164781])[0]]

,name,gender,occupation,age,tenure,cust_id,occ_wealth,occ_animal,occ_int,label,named_trafficker
125454,DAVID WILKERSON,male,Unknown,33.0,0.0,CUST85769551,0,0,0,0,0.0


In [20]:
kdf[kdf['named_trafficker']==1]

,name,gender,occupation,age,tenure,cust_id,occ_wealth,occ_animal,occ_int,label,named_trafficker
73373,VANESSA RONDEAU,female,Other,29.0,4.0,CUST99590442,0,0,0,0,1.0
96713,RYAN MILLAR,male,Unknown,44.0,3.0,CUST23634360,0,0,0,0,1.0
100129,GILLES ALLAIN,male,Unknown,20.0,4.0,CUST44705724,0,0,0,0,1.0
100845,OLIVIA TERRANCE,female,Unknown,38.0,11.0,CUST96671254,0,0,0,0,1.0
141890,XIONG CHANG,male,Cartoonist,20.0,0.0,CUST86859199,0,0,0,0,1.0
143719,GREG ANDERSON,male,Historian,60.0,8.0,CUST74861302,0,0,0,0,1.0
149182,GREGORY LOGAN,male,Unemployed,65.0,8.0,CUST14701697,0,0,0,0,1.0
170931,CHARLOTTE JONES,female,School Bus Driver,67.0,10.0,CUST62443175,0,0,0,0,1.0
187972,ZHOU HONG XIA,male,Unemployed,31.0,4.0,CUST54548565,0,0,0,0,1.0
195693,BRYAN BERG,male,Psychologist,39.0,3.0,CUST76838596,0,0,0,0,1.0
